In [4]:
import os
import sys
spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()
import pyspark

In [5]:
number_cores = 6
memory_gb = 16
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))
sc = pyspark.SparkContext(conf=conf)

In [6]:
!dir /users/trush/CSC496/FrequentItemsets/data

links.csv  movies.csv  ratings.csv  README.txt	tags.csv


In [7]:
ratings = sc.textFile("/users/trush/CSC496/FrequentItemsets/data/ratings.csv")

In [ ]:
ratings.take(5)

In [ ]:
ratings.cache()

In [8]:
%%time
ratings.count()

CPU times: user 26.6 ms, sys: 13.2 ms, total: 39.9 ms
Wall time: 2.05 s


100837

In [9]:
%%time
ratings.count()

CPU times: user 19.1 ms, sys: 7.62 ms, total: 26.7 ms
Wall time: 480 ms


100837

In [10]:
%%time
ratings.count()

CPU times: user 14.7 ms, sys: 11.3 ms, total: 26 ms
Wall time: 282 ms


100837

## Question
- What movies are often reviewed together (hence watched together)?

In [12]:
ratings.getNumPartitions()

2

In [15]:
ratingHeader = ratings.first()
print(ratingHeader)

userId,movieId,rating,timestamp


In [16]:
ratingsOnly = ratings.filter(lambda x : x != ratingHeader)
ratingsOnly.take(5)

['1,1,4.0,964982703',
 '1,3,4.0,964981247',
 '1,6,4.0,964982224',
 '1,47,5.0,964983815',
 '1,50,5.0,964982931']

In [17]:
baskets = ratingsOnly.map(lambda x : x.split(',')) \
        .map(lambda x: (int(x[0]),int(x[1]))) \
        .groupByKey().map(lambda x : list(set(x[1])))
baskets.take(5)

[[115713,
  122882,
  48516,
  91529,
  80906,
  91658,
  114060,
  131724,
  77455,
  79132,
  106782,
  1704,
  112552,
  99114,
  89774,
  109487,
  68157,
  318,
  58559,
  86345,
  333,
  3578,
  60756,
  74458,
  6874,
  8798,
  80489,
  71535,
  46970],
 [1025,
  3079,
  3591,
  2571,
  3083,
  21,
  1046,
  2583,
  4121,
  538,
  539,
  2076,
  2078,
  32,
  1057,
  4641,
  1060,
  2599,
  553,
  1580,
  45,
  47,
  4144,
  1073,
  52,
  1077,
  1079,
  1080,
  58,
  1084,
  1597,
  1086,
  2109,
  2628,
  1094,
  4166,
  588,
  1103,
  593,
  1617,
  595,
  599,
  3160,
  608,
  2145,
  2150,
  3175,
  3176,
  1641,
  106,
  1136,
  2683,
  125,
  126,
  2174,
  4226,
  2692,
  3204,
  4741,
  648,
  2186,
  4239,
  4246,
  2712,
  1179,
  2203,
  2204,
  4252,
  1183,
  4765,
  162,
  1188,
  4260,
  1704,
  171,
  1196,
  1197,
  1198,
  1199,
  176,
  4273,
  1203,
  1719,
  3255,
  1211,
  1213,
  190,
  1219,
  708,
  1732,
  1733,
  1734,
  1225,
  2762,
  2763,
  3788,


In [18]:
numPartitions = baskets.getNumPartitions()
print(numPartitions)

2


We can't use standard mapping to items because that means one single basket at a time. To refer to idea of __local__ itemsets, we will use mapPartitions

In [23]:
rdd = sc.parallelize([1,2,3,4],2)
def f(iterator): yield sum(iterator)
rdd.mapPartitions(f).collect()

[3, 7]

In [27]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9],3)
def f(iterator): yield sum(iterator)
rdd.mapPartitions(f).collect()

[6, 15, 24]

In [28]:
# set support level
support = 100

In [30]:
import sys
import numpy as np
import time